In [ ]:
%matplotlib inline

import gc
import re
import nltk
from  nltk import pos_tag
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

import langdetect
import warnings

warnings.filterwarnings('ignore')

# code starts here!
stop_words_en = set(stopwords.words("english"))
stemmer_en = SnowballStemmer('english')

SOLUTION_FOR_EVERYTHING = 42

# %env CUDA_VISIBLE_DEVICES='0'

# Data Exploration and Pre-processing

## Size of the dataset


In [ ]:
labelled_dataset = pd.read_pickle("data/labelled_dataset.pickle")

labelled_dataset = labelled_dataset.truncate(after=200)

labelled_dataset.shape[0]

**Sample**

In [ ]:
labelled_dataset.head()

In [ ]:
labelled_dataset.describe()

In [ ]:
labelled_dataset.labelmax.value_counts()

In [ ]:
labelled_dataset = labelled_dataset[labelled_dataset.labelmax != "null"]
labelled_dataset.describe()

In [ ]:
classes_array = sorted(list(labelled_dataset.labelmax.unique()))
classes_array

In [ ]:
labelled_dataset = labelled_dataset.drop_duplicates()
labelled_dataset.describe()

## Removing non-English texts
By chance, I found that are also texts in German, French, Spanish, and perhaps in some other languages. For the sake of performance, it will only be trained (and accept) English texts. Further improvements can change this.

There are some Python libraries that can help us on the task of langauge classification:

In [ ]:
def text_language(text):
    return langdetect.detect(text)
    
labelled_dataset = labelled_dataset[labelled_dataset["text"].apply(text_language) == "en"]
labelled_dataset.describe()

In [ ]:
labelled_dataset.labelmax.value_counts()

## Segmentation + Tokenization + Stemming + Stop-word removal

Here was concentrated most of the work of this challenge: preparing the text in order to be processed.

Just an additional **trick** added here. Appart from the standard NLP pre-propressing tasks, as the segments of a review text feature the positive and negative aspects of the review, I hypothesize that textual expressions from each of these segments (pros/cons) may have a different influence to the type company culture determination.

For that reason, appart from segmentate a text into pros/cons parts, I'm also adding the text with a prefix (+/-) 

In [ ]:
# titles of the sehments
SEGMENT_TITLES = '(Pros |Cons |Advice to Management )'

In [ ]:
# returns a segment by it's index in a string splited by SEGMENT_TITLES (i.e.: pros= 2, cons = 4, ...)
def filter_by_segment(entry, index):
    segments = re.split(SEGMENT_TITLES, entry)
    return segments[index]

In [ ]:
# performs tokenisation + stemming + stop-word removal
def tok_stem_stop(segment):
    # tokenize and remove empty entries
    tokens = nltk.word_tokenize(segment.lower())
    # stop-word removal, single character tokens removal, and stemming
    tokens = [stemmer_en.stem(_token) for _token in tokens
              if _token not in stop_words_en
              and len(_token) > 1
              and _token not in ["'s", "'d"]
             ]
    # n't -> not
    tokens = [_token.replace("n't", "not") for _token in tokens]
    return " ".join(tokens)

In [ ]:
# performs tokenisation + gather just words with POS NN/JJ/VB* + stemming
def tok_pos_stem(segment):
    # tokenize and remove empty entries
    tokens = nltk.word_tokenize(segment.lower())
    # choosing only nouns, verbs, and adjectives
    tokens = [_pos[0] for _pos in pos_tag(tokens) if _pos[1] in ["NN", "NNS", "JJ", "VB", "VBG"]]
    # stemming
    tokens = [stemmer_en.stem(_token) for _token in tokens]
    return " ".join(tokens)

In [ ]:
# performs tokenisation + lower case
def tokens(segment):
    return nltk.word_tokenize(segment.lower())

In [ ]:
# performs tokenisation + stop-word removal + lower case
def token_stop(segment):
    tokens = nltk.word_tokenize(segment.lower())
    tokens = [_token for _token in tokens
              if _token not in stop_words_en
              and len(_token) > 1
              and _token not in ["'s", "'d"]
             ]
    return tokens

In [ ]:
# adds a prefix (e.g.: +/-) to a token
def prefix(entry, _prefix):
    return " ".join([_prefix + _token for _token in entry.split()])

### Adding columns with pre-processed data to the dataframe

In [ ]:
labelled_dataset["pros"] = labelled_dataset["text"].apply(lambda entry: filter_by_segment(entry, 2));
labelled_dataset["cons"] = labelled_dataset["text"].apply(lambda entry: filter_by_segment(entry, 4));
labelled_dataset["all"] = labelled_dataset["pros"] + " " + labelled_dataset["cons"];
# labelled_dataset["advice"] = labelled_dataset["text"].apply(lambda entry: filter_by_segment(entry, 6));

In [ ]:
labelled_dataset["pros_tokens"] = labelled_dataset["pros"].apply(token_stop);
labelled_dataset["cons_tokens"] = labelled_dataset["cons"].apply(token_stop);
labelled_dataset["all_tokens"] = labelled_dataset["all"].apply(token_stop);

In [ ]:
labelled_dataset["pros_tok_stem_stop"] = labelled_dataset["pros"].apply(tok_stem_stop);
labelled_dataset["pros_tok_stem_stop"] = labelled_dataset["pros_tok_stem_stop"].apply(lambda entry: prefix(entry, "+"));
labelled_dataset["cons_tok_stem_stop"] = labelled_dataset["cons"].apply(tok_stem_stop);
labelled_dataset["cons_tok_stem_stop"] = labelled_dataset["cons_tok_stem_stop"].apply(lambda entry: prefix(entry, "-"));

labelled_dataset["all_tok_stem_stop"] = labelled_dataset["pros_tok_stem_stop"] + " " + \
    labelled_dataset["cons_tok_stem_stop"]  

In [ ]:
labelled_dataset["pros_tok_pos_stem"] = labelled_dataset["pros"].apply(tok_pos_stem);
labelled_dataset["pros_tok_pos_stem"] = labelled_dataset["pros_tok_pos_stem"].apply(lambda entry: prefix(entry, "+"));
labelled_dataset["cons_tok_pos_stem"] = labelled_dataset["cons"].apply(tok_pos_stem);
labelled_dataset["cons_tok_pos_stem"] = labelled_dataset["cons_tok_pos_stem"].apply(lambda entry: prefix(entry, "-"));

labelled_dataset["all_tok_pos_stem"] = labelled_dataset["pros_tok_pos_stem"] + " " + \
    labelled_dataset["cons_tok_pos_stem"]  

In [ ]:
labelled_dataset.head()

## Reviews length

Tha mnajority of the reviews contain less than 140 words.
**Pros** use to be longer than **cons** (people uses to complain less on Glassdoor?)

In [ ]:
str_lens = labelled_dataset["all_tokens"].apply(len)
plt.xlim(0, 200)
sns.boxplot(str_lens, palette="Set3", showfliers=False);

In [ ]:
str_lens = labelled_dataset["pros_tokens"].apply(len)
plt.xlim(0, 200)
sns.boxplot(str_lens, palette="Set2", showfliers=False);

In [ ]:
str_lens = labelled_dataset["cons_tokens"].apply(len)
plt.xlim(0, 200)
sns.boxplot(str_lens, palette="Set1", showfliers=False);

## Train / Test datasets

In this exploratory study, I'm going to split the dataset into 80% training and 20% testing. It must be ensured that the same class proportions are kept in the test dataset.

In [ ]:
COLUMNS = ["text", "labelmax", "pros", "cons", "all", "pros_tokens", "cons_tokens", "all_tokens", "pros_tok_stem_stop", "cons_tok_stem_stop", "all_tok_stem_stop", "pros_tok_pos_stem", "cons_tok_pos_stem", "all_tok_pos_stem"]

df_train, df_test, _, _ = train_test_split(
    labelled_dataset[COLUMNS],
    labelled_dataset[["labelmax"]],
    stratify=labelled_dataset[["labelmax"]],
    test_size=0.20,
    random_state=SOLUTION_FOR_EVERYTHING
);

## Balancing classes

Classes are not balanced and it may result in a skewed classification based in this data. One solution for this is to oversample minorty classes. As **detail** and **integrity** classes have small subsets, oversampling can also result in skewed results. I'm taking the risk and oversampling all of them:

In [ ]:
sns.countplot(data=df_train, x="labelmax", palette="Set3");

In [ ]:
for _label in set(df_train.labelmax.values):
    ballanced_label_df = resample(
        df_train[df_train.labelmax == _label],
        replace=True,
        n_samples=max(df_train.labelmax.value_counts()),
        random_state=42
    )
    df_train = df_train[df_train.labelmax != _label]
    df_train = pd.concat([df_train, ballanced_label_df])

df_train = df_train.sample(frac=1).reset_index(drop=True)

In [ ]:
sns.countplot(data=df_train, x="labelmax", palette="Set3");

# Classical TF-IDF approach

The classical approach for text classification uses TF-IDF. In this case, I'm using a SVM instead of a Naïve-Bayes classifier.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline

In [ ]:
def tfidf():
    count_vectorizer = CountVectorizer(ngram_range=(1, 1), max_features=500)
    tf_idf = TfidfTransformer(norm="l2")
    svm_clf = SVC()
    tfidf_clf = Pipeline([
        ('count_vectorizer', count_vectorizer),
        ('tf_idf', tf_idf),
        ('svm_clf', svm_clf)
    ])
    return tfidf_clf

In [ ]:
text_clf = tfidf()

In [ ]:
train_data = df_train.as_matrix(columns=["all_tok_pos_stem"])[:,0]
train_labels = df_train.as_matrix(columns=["labelmax"])[:,0]
text_clf.fit(train_data, train_labels);

In [ ]:
predictions = text_clf.predict(df_test.as_matrix(columns=["all_tok_pos_stem"])[:,0])
labels = list(df_test.labelmax)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(labels, predictions)
sns.heatmap(conf_mat, annot=True, fmt='d', cmap="BuPu",
            xticklabels=df_test.labelmax.unique(), yticklabels=df_test.labelmax.unique());

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(labels, predictions, normalize=True)

# CNN (3+3+2) approach

CNNs are among the state-of-the-art of text classification. This is the most well-known CNN implementation (Yoon Kim 2014) for this type of task:

In [ ]:
import gensim
from gensim.models.keyedvectors import Word2VecKeyedVectors
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Embedding, Dense, Convolution1D, ThresholdedReLU, MaxPooling1D, Dropout, Flatten

from keras.callbacks import ModelCheckpoint, EarlyStopping, LambdaCallback

from keras import backend as K

In [ ]:
# word vectors from Glove, imported as gensim keyvecs
wv = Word2VecKeyedVectors.load("models/wv/word.vectors")
weights = np.load("models/wv/word.vectors.vectors.npy")

In [ ]:
SEQUENCE_DIM = 200 # fulfills most of the reviews with 140 words
CLASSES_DIM = len(classes_array) # 6
DROPOUT = 0.1 # maybe...

In [ ]:
# turn labels into categorical arrays
train_labelmax = [classes_array.index(row) for row in df_train["labelmax"].as_matrix()]
test_labelmax = [classes_array.index(row) for row in df_test["labelmax"].as_matrix()]

train_cat_labels = to_categorical(np.asarray(train_labelmax))
test_cat_labels = to_categorical(np.asarray(test_labelmax))

In [ ]:
def create_embedding_vectors(df):
    
    def vectorise_token(token, wv):
        # just ignoring OOVs
        vector = None
        if token in wv.vocab:
            vector = wv.index2entity.index(token)
        return vector
        
    embedding_vectors = []
    
    for review in df:
        review_embedding = []
        for token in review:
            vector = vectorise_token(token, wv)
            if vector is not None:
                review_embedding.append(vector)
        embedding_vectors.append(review_embedding)

    return embedding_vectors

train_embedding = create_embedding_vectors(df_train["all_tokens"])
test_embedding = create_embedding_vectors(df_test["pros_tokens"])

In [ ]:
train_vectors = pad_sequences(train_embedding, maxlen=SEQUENCE_DIM, padding='post')
test_vectors = pad_sequences(test_embedding, maxlen=SEQUENCE_DIM, padding='post')

In [ ]:
# build the convnet here!
def convnet(weights, sequence_dim, classes_dim, dropout_val):
    _lexicon_dim = weights.shape[0]
    _embed_dim = weights.shape[1]
    inputs = Input(shape=(sequence_dim,), dtype='int32')
    embed = Embedding(input_dim=_lexicon_dim, output_dim=_embed_dim, weights=[weights])(inputs)

    x = Convolution1D(100, 5, activation='relu')(embed)
    x = MaxPooling1D(5)(x)
    x = Convolution1D(100, 5, activation='relu')(x)
    x = MaxPooling1D(4)(x)
    x = Convolution1D(100, 5, activation='relu')(x)
    x = MaxPooling1D(3)(x)

    flatten = Flatten()(x)
    dropout = Dropout(dropout_val)(flatten)
    x = Dense(100, activation='relu')(dropout)

    outputs = Dense(classes_dim, activation='softmax', name="last_layer")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = convnet(weights, SEQUENCE_DIM, CLASSES_DIM, DROPOUT)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=1e-4, epsilon=1e-08), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# let's make it faster and avoid the over-fitting that perhaps was arlready happening
early_stop_cb = EarlyStopping(patience=7, monitor='val_acc', mode='max')
checkpoint_cb = ModelCheckpoint("cnn.h5", monitor='val_acc', save_best_only=True, mode='max', verbose=0)

callbacks = [early_stop_cb, checkpoint_cb]

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
train_vectors.shape

In [ ]:
history = model.fit(train_vectors, train_cat_labels, callbacks=callbacks, epochs=100, validation_split=0.10, shuffle=True, batch_size=50)

Validation accuracy:

In [ ]:
history.history["val_acc"][-1]

In [ ]:
h = {"val_acc": history.history["val_acc"], "epoch": history.epoch}

sns.pointplot(x="epoch", y="val_acc", data=pd.DataFrame.from_dict(h));

Test loss and accuracy:

In [ ]:
model.evaluate(test_vectors, test_cat_labels)

In [ ]:
del model
gc.collect()
K.clear_session()

In [ ]:
K.clear_session()

# Results and discussion

The performance of 3 models is evaluated and compared.

In the setup of this experiment, a SVM with stemming and stop-word removal performs better than the other models.

It is statistical significant that CNN performs worse than SVMs in these setup conditions.

In [ ]:
import scipy.stats as st

In [ ]:
NUM_MODELS = 20
P = 0.05
results_array = []

### Evaluating TF-IDF w/ tok+pos+stem

In [ ]:
# dataset for tfidf+tok+pos+stem
train_data = df_train.as_matrix(columns=["all_tok_pos_stem"])[:,0]
train_labels = df_train.as_matrix(columns=["labelmax"])[:,0]
test_data = df_test.as_matrix(columns=["all_tok_pos_stem"])[:,0]
test_labels = list(df_test.labelmax)

In [ ]:
for _ in range(0, 1):
    clf = tfidf()
    clf.fit(train_data, train_labels);
    predictions = clf.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions, normalize=True)
    result = {"model": "tfidf+tok+pos+stem", "acc": accuracy}
    results_array.append(result)

### Evaluating TF-IDF w/ tok+stem+stop

In [ ]:
# dataset for tfidf+tok+stem+stop
train_data = df_train.as_matrix(columns=["all_tok_stem_stop"])[:,0]
train_labels = df_train.as_matrix(columns=["labelmax"])[:,0]
test_data = df_test.as_matrix(columns=["all_tok_stem_stop"])[:,0]
test_labels = list(df_test.labelmax)

In [ ]:
for _ in range(0, 1):
    clf = tfidf()
    clf.fit(train_data, train_labels);
    predictions = clf.predict(test_data)
    accuracy = accuracy_score(test_labels, predictions, normalize=True)
    result = {"model": "tfidf+tok+stem+stop", "acc": accuracy}
    results_array.append(result)

### Evaluating CNN

In [ ]:
for _ in range(0, 10):
    model = convnet(weights, SEQUENCE_DIM, CLASSES_DIM, DROPOUT)
    model.compile(optimizer=Adam(lr=1e-4, epsilon=1e-08), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_vectors, train_cat_labels, epochs=1, validation_split=0.10, shuffle=True, batch_size=50, verbose=0)
    
    accuracy = model.evaluate(test_vectors, test_cat_labels)[1];
    result = {"model": "cnn", "acc": accuracy}
    results_array.append(result)
    del model
    gc.collect()
    K.clear_session()

In [ ]:
sns.boxplot(x="model", y="acc", data=pd.DataFrame.from_dict(results_array), palette="PRGn")

In [ ]:
gc.collect();

### Statistical significance

In [ ]:
[result["acc"] for result in results_array if result["model"] == "tfidf+tok+pos+stem"]

In [ ]:
[result["acc"] for result in results_array if result["model"] == "tfidf+tok+stem+stop"]

In [ ]:
distr = [result["acc"] for result in results_array if result["model"] == "cnn"]
st.t.interval(0.95, len(distr) - 1, loc=np.mean(distr), scale=st.sem(distr))